In [ ]:
import netgen.gui
from ngsolve import *
import scipy.sparse as sp
from ngsolve import internal
from netgen.geom2d import *
import numpy as np
import matplotlib.pyplot as plt

import ipywidgets as widgets
from IPython.display import display
import math 
import numpy.linalg as LA

In [ ]:
def setang(alpha,d):
    if alpha > 90 or alpha < 30:
        return print("wrong angle")
    return d/(np.sin(np.radians(alpha)))

In [ ]:
maxind = 120
mode = "equi"
s = 40

# set the minimumangle
alpha1 = 40
# set the maxangle
alpha2 = 70

In [ ]:
def ChebPoints(a,b,n):
    pi = 3.1415
    x_i = []
    for i in range(0,n+1):
        fac = (2*i+1)/(2*n+2)
        x_ch = np.cos(pi*fac)
        x_i.append((a+b)/2+((b-a)/2)*x_ch)
    return x_i

In [ ]:
geo = SplineGeometry()
geo.AddRectangle ( (0,0), (1,1), bcs=["bottom","right","top","left"], leftdomain=1)

n = 3
r = 0.005
#geo.AddRectangle((0.45,0.49),(0.5, 0.51), bc="interface",leftdomain=0, rightdomain=1)

#geo.AddRectangle((0.45,0.48),(0.5, 0.001), bc="interface",leftdomain=0, rightdomain=1)
#geo.AddRectangle((0.45,0.52),(0.5, 0.999), bc="interface",leftdomain=0, rightdomain=1)


#geo.AddCircle ( (0.5,0.48), r=r, bc="interface",leftdomain=0, rightdomain=1)
#geo.AddCircle ( (0.5,0.52), r=r, bc="interface",leftdomain=0, rightdomain=1)
#geo.AddCircle ( (0.5,0.5), r=r, bc="interface",leftdomain=0, rightdomain=1)
#geo.AddCircle ( (0.5,(1)/(3)), r=r, bc="interface",leftdomain=0, rightdomain=1)
                     
geo.SetMaterial(1, "medium")
# geo.SetMaterial(2, "dot")

mesh = Mesh(geo.GenerateMesh(maxh=0.02))
mesh.Curve(3)
Draw (mesh)

In [ ]:
geo = SplineGeometry()
geo.AddRectangle ( (0,0), (1,1), bcs=["bottom","right","top","left"], leftdomain=1)

r = 0.005
for i in range(s-1):
    geo.AddCircle ( (0.5,(i+1)/(s)), r=r, bc="interface", 
                     leftdomain=0, rightdomain=1)
geo.SetMaterial(1, "medium")


mesh = Mesh(geo.GenerateMesh(maxh=0.02))
mesh.Curve(3)
Draw (mesh)

In [ ]:
source = exp(-10**2*(y-0.5)*(y-0.5))
Draw(source,mesh,name="sorce")

In [ ]:
n = 20
x_i = ChebPoints(-1,1,n)
d = []
for i in range(n+1):
    d.append(0)
plt.plot(x_i,d,"rx")

In [ ]:
lower = setang(alpha2,s)
upper = setang(alpha1,s)
if mode == "Cheb":
    omegfact = np.sort(ChebPoints(lower,upper,maxind))
if mode == "equi":
    omegfact = np.linspace(lower,upper,maxind)

print(mode)
fes = H1(mesh, order=5, complex=True, dirichlet="interface")
gfu = GridFunction(fes)
erg = np.zeros([len(gfu.vec),maxind],dtype=complex)
n = { "medium" : 1, "dot" : 3 }
cfn = CoefficientFunction( [n[mat] for mat in mesh.GetMaterials()] )


u = fes.TrialFunction()
v = fes.TestFunction()

A = BilinearForm(fes)
R = BilinearForm(fes)
K = BilinearForm(fes)
M = BilinearForm(fes)
F = LinearForm(fes)

omega = Parameter(1)
A += (grad(u)*grad(v)- omega**2*cfn*cfn*u*v)*dx
A += -1j*omega*u*v*ds("left|right|top|bottom")
K += grad(u)*grad(v)*dx
R += -1j*u*v*ds("left|right|top|bottom")
M += cfn*cfn*u*v*dx

F = LinearForm(fes)

F += source*v*ds("left")

o = []
norm = []
for i in range(maxind):
    o.append(2*math.pi*omegfact[i])
    #o.append(i)
#o = [2*pi*50]
j = 0
for i in o:   
    with TaskManager():
        omega.Set(i) 
        A.Assemble()
        K.Assemble()
        M.Assemble()
        F.Assemble()
        R.Assemble()   
        inv = A.mat.Inverse(fes.FreeDofs(), inverse="sparsecholesky")
        gfu.vec.data = inv * F.vec
        erg[:,j] = gfu.vec
        fh = F.vec
        norm.append(sqrt (Integrate ((Conj(gfu))*(gfu),mesh)))
        print("Number of iter: ",j,"omega: ",i,"L2-Norm:",sqrt (Integrate ((Conj(gfu))*(gfu),mesh)))
        j = j+1
    
Draw (gfu,mesh,name = "u_h")


rows,cols,vals = M.mat.COO()
Mh = sp.csr_matrix((vals,(rows,cols)))
rows,cols,vals = K.mat.COO()
Kh = sp.csr_matrix((vals,(rows,cols)))
rows,cols,vals = R.mat.COO()
Rh = sp.csr_matrix((vals,(rows,cols)))


In [ ]:
#V, r = np.linalg.qr(erg)
#V = V.dot(np.linalg.inv(r))
V, r = np.linalg.qr(erg)
diff = V.dot(r)-erg
count = 0
for i in range(len(V)):
    for j in range(maxind):
        if diff[i,j] < 1e-14:
            count = count + 1
        else:
            print(i,j,diff[i,j])

print(maxind,"Anzahl an elementen die nicht kleiner als 1e-14 sind: ",np.abs(count-len(V)*maxind))

In [ ]:
MN = np.transpose(V).dot(Mh.dot(V))
KN = np.transpose(V).dot(Kh.dot(V))
RN = np.transpose(V).dot(Rh.dot(V))
fN = np.transpose(V).dot(fh)

In [ ]:
ured = GridFunction(fes)
norm_s =[]
Residual_s=[]
#ngs_temp = K.mat.CreateColVector()
#ngs_temp2 = K.mat.CreateColVector()
uN_s = np.zeros([maxind,maxind],dtype=complex)
j = 0
#print(KN-1**2*MN+1*RN)
for fre in o:
    ngs_temp = K.mat.CreateColVector()
    ngs_temp2 = K.mat.CreateColVector()
    uN_s[:,j] = np.linalg.solve(KN-fre**2*MN+fre*RN, fN)
    print("Omega: ",fre)
    norm_s.append(np.linalg.norm(uN_s[:,j]))

    ured.vec.FV().NumPy()[:] = uN_s[:,j].dot(np.transpose(V))
    with TaskManager():
        ngs_temp.data = K.mat*ured.vec - fre*fre*M.mat*ured.vec + fre * R.mat*ured.vec - F.vec
        ngs_temp2.data = Projector(fes.FreeDofs(), True)*ngs_temp                                      
        res = Norm(ngs_temp2)
        Residual_s.append(res)
    j = j + 1
Residual_s    

In [ ]:
plt.plot(o,Residual_s,label="Norm {}".format(LA.norm(Residual_s)))
plt.grid()
plt.xlabel("omega")
plt.ylabel("res")
plt.title("Residual_snapshots_{}_{}".format(maxind,mode))
plt.legend()
plt.savefig("Residual_snapshots_{}_{}_nonInv".format(maxind,mode))

In [ ]:
u_N_s = GridFunction(fes)
u_N_s.vec.FV().NumPy()[:] = uN_s[:,maxind-1].dot(np.transpose(V))
Draw(u_N_s,mesh,name="u_N_s")
Draw(u_N_s-gfu,mesh,name="u_err")

In [ ]:
ured = GridFunction(fes)
iteration = int(1000)
Start = o[0] 
Ende = o[maxind-1]
lam = []
factor = (Ende-Start) / iteration
omegaN = []
norm_L2 =[]
Residual=[]
ngs_temp = K.mat.CreateColVector()
ngs_temp2 = K.mat.CreateColVector()
uN = np.zeros([maxind,iteration],dtype=complex)
if mode == "Cheb":
    omegaN = np.sort(ChebPoints(Start,Ende,iteration))
for i in range(iteration):
    if mode == "equi":
        omegaN.append(Start+factor*i)
    uN[:,i] = np.linalg.solve(KN-omegaN[i]**2*MN+omegaN[i]*RN, fN)
    print("i: ",i,"Omega: ",omegaN[i])
    norm_L2.append(sqrt(np.vdot(uN[:,i],uN[:,i])))

    ured.vec.FV().NumPy()[:] = uN[:,i].dot(np.transpose(V))
    with TaskManager():
        ngs_temp.data = K.mat*ured.vec - omegaN[i]*omegaN[i]*M.mat*ured.vec + omegaN[i] * R.mat*ured.vec - F.vec
        ngs_temp2.data = Projector(fes.FreeDofs(), True)*ngs_temp                                      
        res = Norm(ngs_temp2)
        Residual.append(res)

In [ ]:
def DrawOmega(amp):
    pi = 3.14159
    #n = 40
    gfu_N = GridFunction(fes)
    gfu_N.vec.FV().NumPy()[:] = uN[:,amp].dot(np.transpose(V))
    ang = np.arcsin((2*pi*40)/omegaN[amp])
    print("omega: ",omegaN[amp])
    print("angle: ",np.degrees(ang))
    Draw (gfu_N,mesh,name="u_N")
    internal.visoptions.autoscale = False
    

In [ ]:
#x = np.linspace(0,10,1000)
amp = widgets.IntSlider(min=0, max=iteration-1, value=4, description = "angle:")
widgets.interactive(DrawOmega,s=s,amp=amp)

In [ ]:
d = []
for i in range(20):
    d.append(0.1)

In [ ]:
plt.semilogy(omegaN[0:iteration],Residual,label="Norm{}".format(LA.norm(Residual)))
plt.grid()
plt.xlabel("omega")
plt.ylabel("res")
plt.title("Residual_iterations_{}_{}".format(maxind,mode))
plt.legend()
plt.savefig("Residual_iterations_{}_{}_nonInv".format(maxind,mode))

In [ ]:
plt.plot(omegaN,norm_L2)
#plt.plot(np.real(l[2:]),d[2:],"bo")

In [ ]:
print(np.arcsin(-1.00000)*45)

In [ ]:
import numpy.linalg as LA
print(LA.norm(Residual))